In [1]:
import os, sys
from os.path import isfile, join, abspath, basename
from optparse import OptionParser
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
import json

pathname = os.path.dirname(sys.argv[0])
full_pathname = os.path.abspath(pathname)
split_pathname = full_pathname.split(sep="mvesc")
base_pathname = os.path.join(split_pathname[0], "mvesc")
parentdir = os.path.join(base_pathname, "ETL")
sys.path.insert(0,parentdir)
from mvesc_utility_functions import *

In [70]:
schema = 'public'
public_tables = ['INV_06_16_CO_M', 'INV_06_16_FR_M', 'INV_06_16_MA_M', 'INV_06_16_RV_M', 'INV_06_16_RW_M', 
                 'INV_06_16_TV_M', 'INV_06_16_WM_M', 'INV_10_16_CEVSD_M', 'INV_10_16_EM_M']
new_column_names = ['student_lookup', 'status', 'grade', 'gender', 'hmrm', 
                    'membership_code', 'description', 'school_year', 'district']

table_df = {}
with postgres_pgconnection_generator() as conn:
    with conn.cursor() as cursor:
        for tab in public_tables:
            nrows = 100
            sql_read_table = """select * from {s}."{t}" limit {n};""".format(s=schema, t=tab, n=nrows)
            table_df[tab] = pd.read_sql_query(sql_read_table, conn)
        table_df['INV_06_16_CO_M']['District'] = 'Coshocton' # add missed `district` column in table for CO

df = pd.DataFrame()
for t in table_df:
    new_col_dict = {table_df[t].columns[i]:new_column_names[i] for i in range(len(new_column_names))}
    table_df[t] = table_df[t].rename(columns=new_col_dict)
    df = df.append(table_df[t])
    #df.drop('hmrm', axis=1, inplace=True)
df = df.drop('hmrm', axis=1)
df.reset_index(drop=True, inplace=True)

,student_lookup,status,grade,gender,membership_code,description,school_year,district
0,17468,A,04,F,152330,Student received intervention provided during ...,8/20/2010,Crooksville
1,17469,A,05,M,152330,Student received intervention provided during ...,8/20/2010,Crooksville
2,17481,A,05,M,152330,Student received intervention provided during ...,10/4/2010,Crooksville
3,17695,A,05,M,152330,Student received intervention provided during ...,8/20/2010,Crooksville
4,18228,I,05,M,152330,Student received intervention provided during ...,8/20/2010,Crooksville
5,18184,A,05,M,152330,Student received intervention provided during ...,8/20/2010,Crooksville
6,54184,A,04,M,152330,Student received intervention provided during ...,10/4/2010,Crooksville
7,18306,A,05,F,152330,Student received intervention provided during ...,8/20/2010,Crooksville
8,18252,A,05,F,152330,Student received intervention provided during ...,10/4/2010,Crooksville
9,8216,NO,04,F,152330,Student received intervention provided during ...,8/20/2010,Crooksville


In [41]:
new_column_names = ['student_lookup', 'status', 'grade', 'hmrm', 
                    'membership_code', 'description', 'school_year', 'district']
{table_df[t].columns[i]:new_column_names[i] for i in range(len(new_column_names))}

{'GD': 'hmrm',
 'GR': 'grade',
 'HMRM': 'membership_code',
 'Membership Description': 'school_year',
 'ST': 'status',
 'STUDENT Membership Code': 'description',
 'StudentLookup': 'student_lookup',
 'YEAR': 'district'}